In [1]:
import pandas as pd 
from pickle import NONE
import mysql.connector
from mysql.connector import Error
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import joblib 

## EDA


In [2]:
data = pd.read_csv('data\ViNames_Train.csv')

In [3]:
data

,Unnamed: 0,Full_Names,Gender
0,0,Ngô Xuân Tùng,0
1,1,Bùi Dương Thảo Vy,1
2,2,Lưu Thế Huy,0
3,3,Nguyễn Thị Vân,1
4,4,Dương Minh Long,0
...,...,...,...
18790,18790,Trần Nguyễn Tường Vy,1
18791,18791,Phan Hải Âu,0
18792,18792,Phạm Trương Trúc Phương,1
18793,18793,Nguyễn Diệu Hiền,1


In [4]:
data_thi = data[(data['Full_Names'].str.contains(' thị ', case = False)) &  (data['Gender'] == 0)].replace(0,1)

In [5]:
test_data = data_thi.combine_first(data)

In [6]:
data = test_data

## Train model

In [7]:
X = data['Full_Names']
Y = data['Gender']

In [8]:
# Split data with 80% for train and 20% for test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state = 0)

In [9]:
# Vectorizer
tf_idf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features = 4000)
tf_idf.fit(X_train)

X_train_enc = tf_idf.transform(X_train) 
X_test_enc = tf_idf.transform(X_test)

In [10]:
# Build SVM model
SVM = LinearSVC()
SVM.fit(X_train_enc, Y_train)

LinearSVC()

In [11]:
#Predict
y_pred = SVM.predict(X_test_enc)

In [12]:
#Evaluation and score
print(classification_report(Y_test.to_list(), y_pred))

              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2152
           1       0.96      0.94      0.95      1607

    accuracy                           0.95      3759
   macro avg       0.95      0.95      0.95      3759
weighted avg       0.95      0.95      0.95      3759



In [13]:
result = pd.DataFrame({
    "Full_Names": X_test, 
    "Giới tính":Y_test,
    "Dự đoán giới tính": y_pred
})
# result.to_excel('ResultForTestTraining.xlsx')

In [14]:
result[(result['Full_Names'].str.contains(' văn ', case = False)) &  (result['Dự đoán giới tính'] == 1)]

,Full_Names,Giới tính,Dự đoán giới tính


In [15]:
#Save model
# joblib.dump(SVM, 'model.joblib')

In [16]:
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='myaloha',
                                         user='root'
                                        )
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL Server version  5.5.5-10.4.24-MariaDB
You're connected to database:  ('myaloha',)


In [17]:
#Query from DB and save data to a DataFrame
query = ("SELECT user_id, name FROM user")
cursor.execute(query)    
myresult = cursor.fetchall()

record = []

for x in myresult:
  record.append(x)


In [18]:
# Get list name from database
list_names = []
user_id = []
for i in range(len(record)):
    user_id.append(record[i][0])
    list_names.append(record[i][1])

In [19]:
df_db = pd.DataFrame({
    "User ID":user_id,
     "Name":list_names 
})

In [20]:
df_db

,User ID,Name
0,1,Lưu Phúc Điền
1,2,test
2,3,Mon
3,4,monamedia
4,5,Vu Phan
...,...,...
99995,116436,Lê Bảo Phúc Nhân
99996,116437,Phạm Nguyễn Hoàng Oanh
99997,116438,Nguyen Thi Hong Ngoc
99998,116439,Shin


In [21]:
#Predict data
name_enc = tf_idf.transform(df_db['Name']) 
model = joblib.load('model.joblib')
predict = model.predict(name_enc)
predict

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [22]:
df_db['Gender predicted'] = predict
# df_db.to_excel('GenderPredicted.xlsx', index = None)

In [23]:
#Các trường hợp chữ lót là Văn và bị đoán sai giới tính
df_db[(df_db['Name'].str.contains(' văn ', case = False)) & (df_db['Gender predicted'] == 1)]

,User ID,Name,Gender predicted
49691,65109,Huỳnh Văn Kim Ngân,1
81650,98083,Phương Thanh Văn Thị,1


## Update data to database

In [38]:
val = []
for i in range(len(df_db)):
    val.append(( int(df_db['Gender predicted'][i]), int(df_db['User ID'][i])))

In [41]:
query = "UPDATE user SET gender =%s WHERE user_id = %s"
cursor.executemany(query, val)
connection.commit()